In [1]:
import torchxrayvision as xrv
import skimage, torch, torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import numpy as np 
import os 
import shutil 
# from utils.utils import log_progress 
import shutil
import pandas as pd
import os
import re
import shutil
np.random.seed(42)
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Load the pre-trained VGG-16 model
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


https://hub.packtpub.com/how-to-leverage-transfer-learning-using-pretrained-cnn-models-tutorial/
- The VGG-16 model is a 16-layer (convolution and fully connected) network built on the ImageNet database, which is built for the purpose of image recognition and classification. 

![VGG-16](https://packt-type-cloud.s3.amazonaws.com/uploads/sites/3149/2018/11/61d69a2a-f31c-477b-9197-b1764c2658b1.png)


In [74]:
df = pd.read_csv('data/CMMD_final.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1104 non-null   object 
 1   right                 1104 non-null   int64  
 2   age                   1104 non-null   float64
 3   abnorm_both           1104 non-null   int64  
 4   abnorm_calcification  1104 non-null   int64  
 5   abnorm_mass           1104 non-null   int64  
 6   class_benign          1104 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 60.5+ KB


,ID,right,age,abnorm_both,abnorm_calcification,abnorm_mass,class_benign
0,0001R,1,0.402985,0,1,0,1
1,0002L,0,0.343284,0,1,0,1
2,0003L,0,0.328358,0,1,0,1
3,0004L,0,0.358209,0,1,0,1
4,0005R,1,0.373134,0,1,0,1


In [75]:
count = (df['class_benign'] == 1).sum()
print(count)

542


In [22]:
1104-542

562

In [77]:
df_benign = df[df['class_benign'] == 1]
df_malignant = df[df['class_benign'] == 0]

df_benign = df_benign.sample(frac=1, random_state=42)
df_malignant = df_malignant.sample(frac=1, random_state=42)


In [78]:
n_train = int(0.8 * len(df))
n_val = int(0.1 * len(df))
n_test = len(df) - n_train - n_val
print(n_train, n_val, n_test)

883 110 111


In [79]:
n_train_benign = int(0.8 * 542)
n_val_benign = int(0.1 * 542)
n_test_benign = 542 - n_train_benign - n_val_benign
print(n_train_benign, n_val_benign, n_test_benign)

433 54 55


In [80]:
n_train_malignant = n_train - n_train_benign
n_val_malignant = n_val - n_val_benign
n_test_malignant = n_test - n_test_benign
print(n_train_malignant, n_val_malignant, n_test_malignant)


450 56 56


In [81]:
train_set = pd.concat([df_benign[:n_train_benign], df_malignant[:n_train_malignant]])
val_set = pd.concat([df_benign[n_train_benign:n_train_benign+n_val_benign],
                     df_malignant[n_train_malignant:n_train_malignant+n_val_malignant]])
test_set = pd.concat([df_benign[-n_test_benign:], df_malignant[-n_test_malignant:]])

In [82]:
train_set

,ID,right,age,abnorm_both,abnorm_calcification,abnorm_mass,class_benign
605,0882R,1,0.343284,0,0,1,1
173,0254R,1,0.402985,1,0,0,1
598,0876L,0,0.402985,0,0,1,1
186,0265L,0,0.194030,1,0,0,1
482,0775L,0,0.432836,0,0,1,1
...,...,...,...,...,...,...,...
364,0647L,0,0.537313,1,0,0,0
799,1074L,0,0.597015,0,0,1,0
918,1346R,1,0.432836,0,0,1,0
827,1140R,1,0.477612,0,0,1,0


In [84]:
train_set.to_csv('data/VGG_training.csv', index=False)
val_set.to_csv('data/VGG_validation.csv', index=False)
test_set.to_csv('data/VGG_testing.csv', index=False) 

In [86]:
# Loop over all the files in the source directory
for filename in os.listdir(r'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG'):
    # Check if the filename matches the pattern 'D1-XXXX.L(R).S(T)'
    if re.match(r'D1-(\d{4})\.(L|R)\.(S|T)', filename):
        # Extract the ID, L/R, and S/T values from the filename
        id_value, lr, st = re.findall(r'D1-(\d{4})\.(L|R)\.(S|T)', filename)[0]
        
        # Construct the new filename in the format 'XXXXL(R)-S(T).jpg'
        new_filename = f'{id_value}{lr}-{st}.jpg'
        
        # Construct the full paths to the source and destination files
        src_file = os.path.join(r'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG', filename)
        dest_file = os.path.join(r'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG', new_filename)
        
        # Rename the file with the new filename
        os.rename(src_file, dest_file)

In [61]:
train_set = pd.read_csv('data/VGG_training.csv')
test_set = pd.read_csv('data/VGG_testing.csv')
val_set = pd.read_csv('data/VGG_validation.csv')

In [89]:
train_set

,ID,right,age,abnorm_both,abnorm_calcification,abnorm_mass,class_benign
605,0882R,1,0.343284,0,0,1,1
173,0254R,1,0.402985,1,0,0,1
598,0876L,0,0.402985,0,0,1,1
186,0265L,0,0.194030,1,0,0,1
482,0775L,0,0.432836,0,0,1,1
...,...,...,...,...,...,...,...
364,0647L,0,0.537313,1,0,0,0
799,1074L,0,0.597015,0,0,1,0
918,1346R,1,0.432836,0,0,1,0
827,1140R,1,0.477612,0,0,1,0


In [90]:
# Define the source and destination directories
src_dir = r'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG'
dest_dir = r'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG/training/training_benign'
# Get a list of unique ID values from the 'ID' column of the 'train_set' DataFrame
id_values = val_set.loc[train_set['class_benign'] == 1, 'ID'].unique()

# Loop over the ID values
for id_value in id_values:
    # Construct the regular expression pattern to match the filenames
    pattern = f'{id_value}-(S|T).jpg'
    
    # Loop over the files in the source directory that match the pattern
    for filename in os.listdir(src_dir):
        if re.match(pattern, filename):
            # Construct the source and destination file paths
            src_file = os.path.join(src_dir, filename)
            dest_file = os.path.join(dest_dir, filename)
            
            # Check if the destination file already exists
            if os.path.exists(dest_file):
                # If the file already exists, append a number to the filename to make it unique
                i = 1
                while os.path.exists(os.path.join(dest_dir, f'{id_value}-{i}.jpg')):
                    i += 1
                dest_file = os.path.join(dest_dir, f'{id_value}-{i}.jpg')
            
            # Move the file to the destination directory
            shutil.move(src_file, dest_file)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [94]:
# Reset the index of the train_set DataFrame
train_set = train_set.reset_index(drop=True)

# Get a list of unique ID values from the 'ID' column of the 'train_set' DataFrame
id_values = train_set.loc[train_set['class_benign'] == 1, 'ID'].unique()

train_set
len(id_values)

433

In [95]:
num_benign = len(train_set[train_set['class_benign'] == 1])
print(num_benign)

433


In [96]:
print(id_values)

['0882R' '0254R' '0876L' '0265L' '0775L' '0257R' '0747L' '1011R' '1040R'
 '1025R' '0007R' '0752L' '0927L' '0256R' '1022R' '0782L' '0995L' '0277L'
 '1034R' '0221L' '0693R' '0892L' '0291L' '0873L' '1021R' '0003L' '0335R'
 '0915L' '0011L' '0698R' '0740L' '0721R' '0831R' '0812L' '1032R' '1035L'
 '0987R' '0254L' '0828L' '0963L' '0994L' '0786R' '0765L' '0739R' '0728R'
 '0259R' '0761R' '0720R' '0829L' '0030L' '0893L' '1020L' '0999L' '0010R'
 '1036L' '0724L' '0279R' '0919L' '0806R' '0828R' '0821R' '0853L' '0257L'
 '1048R' '0252R' '0997R' '0973R' '0891R' '0834R' '0879R' '0258R' '0717L'
 '0786L' '0751R' '0250R' '0861L' '0246R' '0759R' '0909R' '0262R' '0774L'
 '0032R' '0263R' '0901L' '0001R' '0012L' '1004R' '0022L' '0975R' '0269L'
 '0268R' '0955R' '0943L' '0932R' '0683L' '0373R' '0700L' '0019L' '0787R'
 '0763R' '0016R' '0703L' '0036L' '1008L' '0694L' '0043L' '0272L' '1000L'
 '1039L' '0830L' '0768R' '1673L' '0908R' '0851L' '0876R' '0936R' '0862L'
 '0686L' '0815R' '1006R' '0736L' '1059L' '0711L' '0

In [97]:
# Set the source and destination directories
src_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG'
dest_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG/training/training_benign'

In [98]:
# Loop over the ID values and move the corresponding images to the destination directory
for id_value in id_values:
    # Construct the filenames for the source images
    filenames = [f for f in os.listdir(src_dir) if id_value in f]

    # Move each file to the destination directory
    for filename in filenames:
        src_file = os.path.join(src_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        shutil.move(src_file, dest_file)

In [99]:
# Get a list of unique ID values from the 'ID' column of the 'train_set' DataFrame
id_values = train_set.loc[train_set['class_benign'] == 0, 'ID'].unique()

# Set the source and destination directories
src_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG'
dest_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG/training/training_malignant'

# Loop over the ID values and move the corresponding images to the destination directory
for id_value in id_values:
    # Construct the filenames for the source images
    filenames = [f for f in os.listdir(src_dir) if id_value in f]

    # Move each file to the destination directory
    for filename in filenames:
        src_file = os.path.join(src_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        shutil.move(src_file, dest_file)

In [102]:
# Get a list of unique ID values from the 'ID' column of the 'train_set' DataFrame
id_values = val_set.loc[val_set['class_benign'] == 1, 'ID'].unique()

# Set the source and destination directories
src_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG'
dest_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG/validation/validation_benign'

# Loop over the ID values and move the corresponding images to the destination directory
for id_value in id_values:
    # Construct the filenames for the source images
    filenames = [f for f in os.listdir(src_dir) if id_value in f]

    # Move each file to the destination directory
    for filename in filenames:
        src_file = os.path.join(src_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        shutil.move(src_file, dest_file)

In [103]:
# Get a list of unique ID values from the 'ID' column of the 'train_set' DataFrame
id_values = val_set.loc[val_set['class_benign'] == 0, 'ID'].unique()

# Set the source and destination directories
src_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG'
dest_dir = 'C:/Users/brigi/Briggy Dropbox/Brigitte Yan/DataScience/breast_cancer_classifier/images/final_images_VGG/validation/validation_malignant'

# Loop over the ID values and move the corresponding images to the destination directory
for id_value in id_values:
    # Construct the filenames for the source images
    filenames = [f for f in os.listdir(src_dir) if id_value in f]

    # Move each file to the destination directory
    for filename in filenames:
        src_file = os.path.join(src_dir, filename)
        dest_file = os.path.join(dest_dir, filename)
        shutil.move(src_file, dest_file)

In [104]:
test_set

,ID,right,age,abnorm_both,abnorm_calcification,abnorm_mass,class_benign
560,0845L,0,0.373134,0,0,1,1
188,0267L,0,0.417910,1,0,0,1
632,0904R,1,0.059701,0,0,1,1
518,0807R,1,0.283582,0,0,1,1
411,0710R,1,0.059701,0,0,1,1
...,...,...,...,...,...,...,...
122,0191L,0,0.597015,0,1,0,0
237,0321L,0,0.417910,1,0,0,0
809,1095L,0,0.582090,0,0,1,0
975,1493R,1,0.611940,0,0,1,0


In [2]:
df = pd.read_csv('data/CMMD_final.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1104 entries, 0 to 1103
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1104 non-null   object 
 1   right                 1104 non-null   int64  
 2   age                   1104 non-null   float64
 3   abnorm_both           1104 non-null   int64  
 4   abnorm_calcification  1104 non-null   int64  
 5   abnorm_mass           1104 non-null   int64  
 6   class_benign          1104 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 60.5+ KB


In [6]:
directory = 'images/final_images_VGG/training/training_benign'
n_train_benign = os.listdir(directory)
train_benign = np.array(n_train_benign)
print(train_benign)

['0001R-S.jpg' '0001R-T.jpg' '0003L-S.jpg' '0003L-T.jpg' '0004L-S.jpg'
 '0004L-T.jpg' '0005R-S.jpg' '0005R-T.jpg' '0006L-S.jpg' '0006L-T.jpg'
 '0007R-S.jpg' '0007R-T.jpg' '0008L-S.jpg' '0008L-T.jpg' '0009L-S.jpg'
 '0009L-T.jpg' '0010R-S.jpg' '0010R-T.jpg' '0011L-S.jpg' '0011L-T.jpg'
 '0012L-S.jpg' '0012L-T.jpg' '0013L-S.jpg' '0013L-T.jpg' '0015R-S.jpg'
 '0015R-T.jpg' '0016R-S.jpg' '0016R-T.jpg' '0017R-S.jpg' '0017R-T.jpg'
 '0018R-S.jpg' '0018R-T.jpg' '0019L-S.jpg' '0019L-T.jpg' '0020R-S.jpg'
 '0020R-T.jpg' '0022L-S.jpg' '0022L-T.jpg' '0023R-S.jpg' '0023R-T.jpg'
 '0024R-S.jpg' '0024R-T.jpg' '0025L-S.jpg' '0025L-T.jpg' '0026R-S.jpg'
 '0026R-T.jpg' '0027L-S.jpg' '0027L-T.jpg' '0028L-S.jpg' '0028L-T.jpg'
 '0029L-S.jpg' '0029L-T.jpg' '0030L-S.jpg' '0030L-T.jpg' '0031R-S.jpg'
 '0031R-T.jpg' '0032L-S.jpg' '0032L-T.jpg' '0032R-S.jpg' '0032R-T.jpg'
 '0034L-S.jpg' '0034L-T.jpg' '0034R-S.jpg' '0034R-T.jpg' '0035L-S.jpg'
 '0035L-T.jpg' '0035R-S.jpg' '0035R-T.jpg' '0036L-S.jpg' '0036L-T.jpg'
 '0037

In [8]:
directory = 'images/final_images_VGG/training/training_malignant'
n_train_malignant = os.listdir(directory)
train_malignant = np.array(n_train_malignant)
print(train_malignant)

['0045L-S.jpg' '0045L-T.jpg' '0047R-S.jpg' '0047R-T.jpg' '0051R-S.jpg'
 '0051R-T.jpg' '0053R-S.jpg' '0053R-T.jpg' '0055R-S.jpg' '0055R-T.jpg'
 '0056L-S.jpg' '0056L-T.jpg' '0058R-S.jpg' '0058R-T.jpg' '0061R-S.jpg'
 '0061R-T.jpg' '0063L-S.jpg' '0063L-T.jpg' '0064L-S.jpg' '0064L-T.jpg'
 '0067R-S.jpg' '0067R-T.jpg' '0071R-S.jpg' '0071R-T.jpg' '0075R-S.jpg'
 '0075R-T.jpg' '0078R-S.jpg' '0078R-T.jpg' '0084L-S.jpg' '0084L-T.jpg'
 '0085R-S.jpg' '0085R-T.jpg' '0091L-S.jpg' '0091L-T.jpg' '0101R-S.jpg'
 '0101R-T.jpg' '0102L-S.jpg' '0102L-T.jpg' '0103L-S.jpg' '0103L-T.jpg'
 '0105R-S.jpg' '0105R-T.jpg' '0106R-S.jpg' '0106R-T.jpg' '0107L-S.jpg'
 '0107L-T.jpg' '0110L-S.jpg' '0110L-T.jpg' '0111L-S.jpg' '0111L-T.jpg'
 '0112R-S.jpg' '0112R-T.jpg' '0113R-S.jpg' '0113R-T.jpg' '0116R-S.jpg'
 '0116R-T.jpg' '0121R-S.jpg' '0121R-T.jpg' '0122R-S.jpg' '0122R-T.jpg'
 '0124R-S.jpg' '0124R-T.jpg' '0127L-S.jpg' '0127L-T.jpg' '0129R-S.jpg'
 '0129R-T.jpg' '0131L-S.jpg' '0131L-T.jpg' '0132R-S.jpg' '0132R-T.jpg'
 '0134

In [10]:
directory = 'images/final_images_VGG/validation/validation_benign'
n_val_benign = os.listdir(directory)
val_benign = np.array(n_val_benign)
print(val_benign)

['0002L-S.jpg' '0002L-T.jpg' '0014L-S.jpg' '0014L-T.jpg' '0033R-S.jpg'
 '0033R-T.jpg' '0039R-S.jpg' '0039R-T.jpg' '0121L-S.jpg' '0121L-T.jpg'
 '0203L-S.jpg' '0203L-T.jpg' '0207R-S.jpg' '0207R-T.jpg' '0260L-S.jpg'
 '0260L-T.jpg' '0270L-S.jpg' '0270L-T.jpg' '0280R-S.jpg' '0280R-T.jpg'
 '0706R-S.jpg' '0706R-T.jpg' '0734R-S.jpg' '0734R-T.jpg' '0744R-S.jpg'
 '0744R-T.jpg' '0748R-S.jpg' '0748R-T.jpg' '0750L-S.jpg' '0750L-T.jpg'
 '0754R-S.jpg' '0754R-T.jpg' '0758L-S.jpg' '0758L-T.jpg' '0759L-S.jpg'
 '0759L-T.jpg' '0774R-S.jpg' '0774R-T.jpg' '0779L-S.jpg' '0779L-T.jpg'
 '0788L-S.jpg' '0788L-T.jpg' '0795R-S.jpg' '0795R-T.jpg' '0799L-S.jpg'
 '0799L-T.jpg' '0800L-S.jpg' '0800L-T.jpg' '0803R-S.jpg' '0803R-T.jpg'
 '0813L-S.jpg' '0813L-T.jpg' '0827L-S.jpg' '0827L-T.jpg' '0833R-S.jpg'
 '0833R-T.jpg' '0839L-S.jpg' '0839L-T.jpg' '0863R-S.jpg' '0863R-T.jpg'
 '0865L-S.jpg' '0865L-T.jpg' '0870R-S.jpg' '0870R-T.jpg' '0874R-S.jpg'
 '0874R-T.jpg' '0881L-S.jpg' '0881L-T.jpg' '0887L-S.jpg' '0887L-T.jpg'
 '0898

In [11]:
directory = 'images/final_images_VGG/validation/validation_malignant'
n_val_malignant = os.listdir(directory)
val_malignant = np.array(n_val_malignant)
print(val_malignant)

['0046R-S.jpg' '0046R-T.jpg' '0059R-S.jpg' '0059R-T.jpg' '0070L-S.jpg'
 '0070L-T.jpg' '0130R-S.jpg' '0130R-T.jpg' '0133R-S.jpg' '0133R-T.jpg'
 '0147R-S.jpg' '0147R-T.jpg' '0152L-S.jpg' '0152L-T.jpg' '0154L-S.jpg'
 '0154L-T.jpg' '0176R-S.jpg' '0176R-T.jpg' '0180L-S.jpg' '0180L-T.jpg'
 '0313R-S.jpg' '0313R-T.jpg' '0367R-S.jpg' '0367R-T.jpg' '0378L-S.jpg'
 '0378L-T.jpg' '0387L-S.jpg' '0387L-T.jpg' '0423L-S.jpg' '0423L-T.jpg'
 '0435R-S.jpg' '0435R-T.jpg' '0436L-S.jpg' '0436L-T.jpg' '0519R-S.jpg'
 '0519R-T.jpg' '0530L-S.jpg' '0530L-T.jpg' '0568R-S.jpg' '0568R-T.jpg'
 '0584R-S.jpg' '0584R-T.jpg' '0597L-S.jpg' '0597L-T.jpg' '0606R-S.jpg'
 '0606R-T.jpg' '0607R-S.jpg' '0607R-T.jpg' '0663L-S.jpg' '0663L-T.jpg'
 '1055L-S.jpg' '1055L-T.jpg' '1071L-S.jpg' '1071L-T.jpg' '1094R-S.jpg'
 '1094R-T.jpg' '1120R-S.jpg' '1120R-T.jpg' '1163L-S.jpg' '1163L-T.jpg'
 '1170R-S.jpg' '1170R-T.jpg' '1178L-S.jpg' '1178L-T.jpg' '1187L-S.jpg'
 '1187L-T.jpg' '1221L-S.jpg' '1221L-T.jpg' '1257R-S.jpg' '1257R-T.jpg'
 '1277

In [16]:
directory = 'images/final_images_VGG/test'
n_test_benign = os.listdir(directory)

In [18]:
train_files = np.concatenate((train_benign, train_malignant))
val_files = np.concatenate((val_benign, val_malignant))
test_files = np.array(n_test_benign)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.3,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/path/to/my/images/train',
    target_size=(150, 150),
    batch_size=30,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    '/path/to/my/images/validation',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical')
